In [6]:
import gradio as gr
import json
import os
# JSON dosyasını oku
def load_json(json_path):
    with open(json_path, "r", encoding="utf-8") as file:
        return json.load(file)

# JSON dosyasını güncelle
def save_json(json_path, data):
    with open(json_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)


In [7]:
JSON_PATH = "output.json"
IMAGE_FOLDER = "images"

# JSON verisini yükle
data = load_json(JSON_PATH)
current_index = 0

# Mevcut resmi ve açıklamaları getir
# Mevcut resmi ve açıklamaları getir
# Mevcut resmi ve açıklamaları getir
def get_current_data():
    global current_index
    item = data[current_index]
    image_path = os.path.join(IMAGE_FOLDER, item["image"])
    counter_text = f"{current_index + 1}/{len(data)}"
    return image_path, item["prompt"], item["chosen"], item["rejected"], counter_text

# Kullanıcının girdisini kaydet
def update_chosen(user_input):
    global current_index
    data[current_index]["chosen"] = user_input
    save_json(JSON_PATH, data)
    return "Yanıt kaydedildi!"

# Resmi değiştir (ileri/geri)
def change_image(direction):
    global current_index
    if direction == "next" and current_index < len(data) - 1:
        current_index += 1
    elif direction == "prev" and current_index > 0:
        current_index -= 1
    return get_current_data()

# Resmi sil
def delete_image():
    global current_index
    item = data[current_index]
    image_path = os.path.join(IMAGE_FOLDER, item["image"])
    
    # Resmi klasörden sil
    if os.path.exists(image_path):
        os.remove(image_path)
    
    # JSON verisinden sil
    del data[current_index]
    save_json(JSON_PATH, data)
    
    # Eğer silinen son resimse önceki resme dön
    if current_index >= len(data):
        current_index -= 1
    
    # Veri kalmadıysa boş değerler döndür
    if len(data) == 0:
        return "Tüm veriler silindi.", "", "", "", "0/0"
    
    return get_current_data()

# Gradio arayüzü
def interface():
    with gr.Blocks() as app:
        with gr.Row():
            image = gr.Image(height=250, width=250)
        with gr.Row():
            prompt = gr.Textbox(label="prompt")
        with gr.Row():
            chosen = gr.Textbox(label="chosen")
            rejected = gr.Textbox(label="rejected")

        with gr.Row():
            user_input = gr.Textbox(label="Yanıtını Gir")
            with gr.Column(scale=0.1):
                 counter = gr.Label(label="Sayaç")  # Sayaç üstte yer alıyor.
        with gr.Row():
            submit_btn = gr.Button("Kaydet")
            prev_btn = gr.Button("Önceki")
            next_btn = gr.Button("Sonraki")
            delete_btn = gr.Button("Resmi Sil")
        
        # Başlangıç verileri
        image_path, prompt_text, chosen_text, rejected_text, counter_text = get_current_data()
        image.value = image_path
        prompt.value = prompt_text
        chosen.value = chosen_text
        rejected.value = rejected_text
        counter.value = counter_text
        prev_text = gr.Text("prev", visible=False)
        next_text = gr.Text("next", visible=False)
        # Buton işlevleri
        submit_btn.click(update_chosen, inputs=[user_input], outputs=[])
        prev_btn.click(change_image, inputs=[prev_text], outputs=[image, prompt, chosen, rejected, counter])
        next_btn.click(change_image, inputs=[next_text], outputs=[image, prompt, chosen, rejected, counter])
        delete_btn.click(delete_image, inputs=[], outputs=[image, prompt, chosen, rejected, counter])
    
    return app

In [8]:
app = interface()
app.launch()

c:\Users\Berke\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\layouts\column.py:55: UserWarning: 'scale' value should be an integer. Using 0.1 will cause issues.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
